In [149]:
from abc import ABC, abstractmethod
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import IterableDataset, DataLoader, random_split
import pytorch_lightning as pl

from contextualized.modules import NGAM, MLP, SoftSelect, Explainer


ENCODERS = {
    'mlp': MLP,
    'ngam': NGAM,
}
MODELS = ['multivariate', 'univariate']
METAMODELS = ['simple', 'subtype', 'multitask', 'tasksplit']
LINK_FNS = [
    lambda x: x,
    lambda x: F.softmax(x, dim=1)
]


class CRTrainer(pl.Trainer):
    def predict_coefs(self, model, dataloader):
        preds = super().predict(model, dataloader)
        return model._coef_preds(preds, dataloader)
    
    def predict_y(self, model, dataloader):
        preds = super().predict(model, dataloader)
        return model._y_preds(preds, dataloader)


class Dataset:
    def __init__(self, C, X, Y, dtype=torch.float):
        """
        C: (n x c_dim)
        X: (n x x_dim)
        Y: (n x y_dim)
        """
        self.C = torch.tensor(C, dtype=dtype)
        self.X = torch.tensor(X, dtype=dtype)
        self.Y = torch.tensor(Y, dtype=dtype)
        self.n_i = 0
        self.x_i = 0
        self.y_i = 0
        self.n = C.shape[0]
        self.c_dim = C.shape[-1]
        self.x_dim = X.shape[-1]
        self.y_dim = Y.shape[-1]
        self.dtype = dtype

    def __iter__(self):
        self.n_i = 0
        self.x_i = 0
        self.y_i = 0
        return self
    
    @abstractmethod
    def __next__(self):
        pass
    
    @abstractmethod
    def __len__(self):
        pass


class MultivariateDataset(Dataset):
    def __next__(self):
        pass
        
    def __len(self):
        return self.n
    
class UnivariateDataset(Dataset):
    def __next__(self):
        pass
        
    def __len(self):
        return self.n
    
class MultitaskMultivariateDataset(Dataset):
    def __next__(self):
        pass
        
    def __len(self):
        return self.n * self.y_dim
    
class MultitaskUnivariateDataset(Dataset):
    def __next__(self):
        if self.n_i >= self.n:
            self.n_i = 0
            raise StopIteration
        t = torch.zeros(self.x_dim + self.y_dim)
        t[self.x_i] = 1
        t[self.x_dim + self.y_i] = 1
        ret = (
            self.C[self.n_i],
            t,
            self.X[self.n_i, self.x_i:self.x_i+1],
            self.Y[self.n_i, self.y_i:self.y_i+1],
            self.n_i,
            self.x_i,
            self.y_i,
        )
        self.y_i += 1
        if self.y_i >= self.y_dim:
            self.x_i += 1
            self.y_i = 0
        if self.x_i >= self.x_dim:
            self.n_i += 1
            self.x_i = 0
        return ret
    
    def __len__(self):
        return self.n * self.x_dim * self.y_dim    

class DataIterable(IterableDataset):
    def __init__(self, dataset):
        self.dataset = dataset
    
    def __iter__(self):
        return iter(self.dataset)


# class ContextualizedRegressionAbstract(pl.LightningModule):
def MSE(beta, mu, x, y, link_fn=lambda x: x):
    y_hat = link_fn((beta * x).sum(axis=1).unsqueeze(-1) + mu)
    residual = y_hat - y
    return residual.pow(2).mean()


class NaiveContextualizedRegression(pl.LightningModule):
    def __init__(self, context_dim, x_dim, y_dim, encoder_type='mlp', univariate=False,
                 num_archetypes=10, encoder_width=25, encoder_layers=2):
        super().__init__()
        self.context_dim = context_dim
        self.x_dim = x_dim
        self.y_dim = y_dim
        self.link_fn = link_fn

        encoder = ENCODERS[encoder_type]
        mu_dim = x_dim if univariate else 1
        out_dim = (x_dim + mu_dim) * y_dim
        self.context_encoder = encoder(context_dim, out_dim, encoder_width, encoder_layers)

    def forward(self, C):
        W = self.context_encoder(C)
        W = torch.reshape(out, (out.shape[0], self.y_dim, x_dim + mu_dim))
        beta = W[:, :, :self.x_dim]
        mu = W[:, :, self.x_dim:]
        return beta, mu


class SubtypeContextualizedRegression(pl.LightningModule):
    def __init__(self, context_dim, x_dim, y_dim, encoder_type='mlp', univariate=False,
                 num_archetypes=10, encoder_width=25, encoder_layers=2, link_fn=lambda x: x):
        super().__init__()
        self.context_dim = context_dim
        self.x_dim = x_dim
        self.y_dim = y_dim
        self.link_fn = link_fn

        encoder = ENCODERS[encoder_type]
        mu_dim = x_dim if univariate else 1
        self.context_encoder = encoder(context_dim, num_archetypes, encoder_width, encoder_layers)
        self.explainer = Explainer(num_archetypes, (self.y_dim, x_dim + mu_dim))

    def forward(self, C):
        Z_pre = self.context_encoder(C)
        Z = self.link_fn(Z_pre)
        W = self.explainer(Z)
        beta = W[:, :, :self.x_dim]
        mu = W[:, :, self.x_dim:]
        return beta, mu


class MultitaskContextualizedRegression(pl.LightningModule):
    def __init__(self, context_dim, x_dim, y_dim, encoder_type='mlp', univariate=False,
                 num_archetypes=10, encoder_width=25, encoder_layers=2, link_fn=lambda x: x):
        super().__init__()
        self.context_dim = context_dim
        self.x_dim = x_dim
        self.y_dim = y_dim
        self.link_fn = link_fn

        encoder = ENCODERS[encoder_type]
        beta_dim = 1 if univariate else x_dim
        task_dim = y_dim + x_dim if univariate else y_dim
        self.context_encoder = encoder(context_dim + task_dim, num_archetypes, encoder_width, encoder_layers)
        self.explainer = Explainer(num_archetypes, (beta_dim + 1, ))

    def forward(self, C, T):
        CT = torch.cat((C, T), 1)
        Z_pre = self.context_encoder(CT)
        Z = self.link_fn(Z_pre)
        W = self.explainer(Z)
        beta = W[:, :-1]
        mu = W[:, -1:]
        return beta, mu

    
class TasksplitContextualizedRegression(pl.LightningModule):
    def __init__(self, context_dim, x_dim, y_dim, encoder_type='mlp', univariate=False,
                 num_archetypes=10, encoder_width=25, encoder_layers=2, link_fn=lambda x: x):
        super().__init__()
        self.context_dim = context_dim
        self.x_dim = x_dim
        self.y_dim = y_dim
        self.link_fn = link_fn
        self.build()

    def build(self):
        encoder = ENCODERS[encoder_type]
        beta_dim = 1 if univariate else x_dim
        task_dim = y_dim + x_dim if univariate else y_dim
        self.context_encoder = encoder(context_dim, num_archetypes, encoder_width, encoder_layers)
        self.task_encoder = encoder(task_dim, num_archetypes, encoder_width, encoder_layers)
        self.explainer = SoftSelect((num_archetypes, num_archetypes), (beta_dim + 1, ))

    def forward(self, C, T):
        Z_c_pre = self.context_encoder(C) 
        Z_t_pre = self.task_encoder(T)
        Z_c = self.link_fn(Z_c_pre)
        Z_t = self.link_fn(Z_t_pre)
        W = self.explainer(Z_c, Z_t)
        beta = W[:, :-1]
        mu = W[:, -1:]
        return beta, mu
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer
    
    def training_step(self, batch, batch_idx):
        C, T, X, Y, _, _, _ = batch
        beta_hat, mu_hat = self(C, T)
        loss = MSE(beta_hat, mu_hat, X, Y)
        return loss
    
    def validation_step(self, batch, batch_idx):
        C, T, X, Y, _, _, _ = batch
        beta_hat, mu_hat = self(C, T)
        loss = MSE(beta_hat, mu_hat, X, Y)
        return loss
    
    def test_step(self, batch, batch_idx):
        C, T, X, Y, _, _, _ = batch
        beta_hat, mu_hat = self(C, T)
        loss = MSE(beta_hat, mu_hat, X, Y)
        self.log_dict({'test_mse': loss})
    
    def predict_step(self, batch, batch_idx):
        C, T, X, Y, _, _, _ = batch
        beta_hat, mu_hat = self(C, T)
        return beta_hat, mu_hat
    
    def _coef_preds(self, preds, dataloader):
        ds = dataloader.dataset.dataset
        betas = np.zeros((ds.n, 
                          ds.x_dim, 
                          ds.y_dim))
        mus = betas.copy()
        for (beta_hats, mu_hats), data in zip(preds, dataloader):
            _, _, _, _, n_idx, x_idx, y_idx = data
            for beta_hat, mu_hat, n_i, x_i, y_i in zip(beta_hats, mu_hats, n_idx, x_idx, y_idx):
                betas[n_i, x_i, y_i] = beta_hat
                mus[n_i, x_i, y_i] = mu_hat
        return betas, mus
    
    def _y_preds(self, preds, dataloader):
        ds = dataloader.dataset.dataset
        ys = np.zeros((ds.n, 
                       ds.x_dim, 
                       ds.y_dim))
        for (beta_hats, mu_hats), data in zip(preds, dataloader):
            _, _, X, Y, n_idx, x_idx, y_idx = data
            for beta_hat, mu_hat, x, y, n_i, x_i, y_i in zip(beta_hats, mu_hats, X, Y, n_idx, x_idx, y_idx):
                ys[n_i, x_i, y_i] = beta_hat * y + mu_hat
        return ys
    
    def dataloader(self, C, X, Y, batch_size=32):
        return DataLoader(dataset=DataIterable(MultitaskUnivariateDataset(C, X, Y)), batch_size=batch_size)
    
class ContextualizedUnivariateRegression:
    def __init__(self, context_dim, x_dim, y_dim, metamodel='tasksplit'):
        if metamodel == 'tasksplit':
            self.model = TasksplitContextualizedRegression(context_dim, x_dim, y_dim, univariate=True)

In [150]:
n = 100
c_dim = 4
x_dim = 2
y_dim = 3
C = torch.rand((n, c_dim)) - .5 
W_1 = C.sum(axis=1).unsqueeze(-1) ** 2
W_2 = - C.sum(axis=1).unsqueeze(-1)
b_1 = C[:, 0].unsqueeze(-1)
b_2 = C[:, 1].unsqueeze(-1)
W_full = torch.cat((W_1, W_2), axis=1)
b_full = b_1 + b_2
X = torch.rand((n, x_dim)) - .5
Y_1 = X[:, 0].unsqueeze(-1) * W_1 + b_1
Y_2 = X[:, 1].unsqueeze(-1) * W_2 + b_2
Y_3 = X.sum(axis=1).unsqueeze(-1)
Y = torch.cat((Y_1, Y_2, Y_3), axis=1)

In [151]:
reg = TasksplitContextualizedRegression(c_dim, x_dim, y_dim, univariate=True)
dataloader = reg.dataloader(C, X, Y, batch_size=32)
trainer = CRTrainer(max_epochs=1)
trainer.fit(reg, dataloader)
beta_preds, mu_preds = trainer.predict_coefs(reg, dataloader)
y_preds = trainer.predict_y(reg, dataloader)
trainer.test(reg, dataloader)
# reg.reshape_preds(preds, dataloader)[0].shape

/var/folders/_6/468mk1cx3cb080z2r5gq0k_r0000gn/T/ipykernel_38844/2009399854.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.C = torch.tensor(C, dtype=dtype)
/var/folders/_6/468mk1cx3cb080z2r5gq0k_r0000gn/T/ipykernel_38844/2009399854.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.X = torch.tensor(X, dtype=dtype)
/var/folders/_6/468mk1cx3cb080z2r5gq0k_r0000gn/T/ipykernel_38844/2009399854.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.Y = torch.tensor(Y, dtype=dtype)
GPU available: False, used: False
TPU avail

Epoch 0: : 20it [00:00, 122.00it/s, loss=0.116, v_num=36]
Predicting: 38it [00:00, 752.14it/s]
(100, 2, 3)
Predicting: 38it [00:00, 672.61it/s]
(100, 2, 3)
Testing: 0it [00:00, ?it/s]0.10918955504894257
0.13371214270591736
0.09991994500160217
0.12591280043125153
0.10867510735988617
0.18073561787605286
0.1461264044046402
0.05753888934850693
0.10406116396188736
0.14782333374023438
0.07600175589323044
0.10068490356206894
0.04651330038905144
0.12310661375522614
0.13345009088516235
0.11001557111740112
0.10752174258232117
0.0946996882557869
0.15984398126602173
--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_loss': 0.1133638247847557}
--------------------------------------------------------------------------------
Testing: 20it [00:00, 400.90it/s]


[{'test_loss': 0.1133638247847557}]


  | Name            | Type       | Params
-----------------------------------------------
0 | context_encoder | MLP        | 385   
1 | task_encoder    | MLP        | 410   
2 | explainer       | SoftSelect | 200   
-----------------------------------------------
995       Trainable params
0         Non-trainable params
995       Total params
0.004     Total estimated model params size (MB)


Epoch 9: : 20it [00:00, 176.42it/s, loss=0.0792, v_num=15]
